In [ ]:
from scipy import stats
from pprint import pprint
import numpy as np
from scipy import spatial
from sklearn.metrics import cohen_kappa_score
import requests

In [ ]:
PATH_VAL = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/annotations_fabio_andrea.txt'
PATH_SYN = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/babel_synset.txt'
PATH_NASARI = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/mini_NASARI.tsv'
PATH_SEM = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/SemEval17_IT_senses2synsets.txt' 

KEY = '5da132f9-c606-4832-9a4f-3c2e3f739cf7'
KEY2 = 'd4bc0872-d933-4516-b456-2063fa7d68ff'
URL = 'https://babelnet.io/v7/getSynset?id={}&key={}&targetLang=IT&searchLang=IT'

#### Recupero risorse da files
Vengono salvati i dati delle nostre annotazioni (sia le valutazioni, sia in synset) in delle liste e vengono recuperate anche le parole da analizzare

In [ ]:
def get_synsets_from_txt(path):
    andrea_list = []
    fabio_list = []
    with open(path, 'r') as file:
        for line in file.readlines():
            andrea_list.append(line.split('    ')[2].strip('\n'))
            andrea_list.append(line.split('    ')[3].strip('\n'))
            fabio_list.append(line.split('    ')[4].strip('\n'))
            fabio_list.append(line.split('    ')[5].strip('\n'))
    
    return andrea_list, fabio_list

def get_valutation_from_txt(path):
    andrea_list = []
    fabio_list = []
    mean_list = []
    with open(path, 'r') as file:
        for line in file.readlines():
            andrea_list.append(float(line.split('    ')[2]))
            fabio_list.append(float(line.split('    ')[3]))
            mean_list.append(float(line.split('    ')[4]))

    return andrea_list, fabio_list, mean_list

def get_words_from_txt(path):
    list_of_words = []
    with open(path, 'r') as file:
        for line in file.readlines():
            list_of_words.append((line.split('    ')[0],line.split('    ')[1]))
    return list_of_words

#### Recupero vettori di NASARI
Viene salvato il file con i vettori NASARI all'interno di un dizionario che ha come chiavi il babelId e come valori gli elementi del vettore associato

In [ ]:
def nasari_to_dict(path):
    nasari_dict = {}
    with open(path, 'r', encoding="utf8") as file:
        for line in file.readlines():
            splitted_line = line.split('\t')
            nasari_dict[splitted_line[0].split('__')[0]] = splitted_line[1:]
            
    return nasari_dict

In [ ]:
words = get_words_from_txt(PATH_VAL)
nasari_dict = nasari_to_dict(PATH_NASARI)

#### Recupero synset da SemEval17_IT_senses2synsets.txt
Da file recupero ogni possibile babelId associato al nostro termine da valutare 

In [ ]:
def get_babel_syns(word, path):
    babel_list = []
    flag = False
    with open(path, 'r', encoding="utf8") as file: 
        for line in file.readlines():
            if word in line:
                flag = True
            elif flag and line[0] != '#':
                babel_list.append(line)
            else:
                flag = False
    return babel_list                

#### Calcolo similarità

In [ ]:
def cos_sim(vec1, vec2):
    return 1 - spatial.distance.cosine(vec1, vec2)

#### Calcolo il massimo tra i valori di similarità

In [ ]:
def get_nasari_valutation(syn1, syn2, nasari_dict):
    max = 0.0
    max_elem = (None, None)
    for s1 in syn1:
        for s2 in syn2:
            if s1.strip('\n') in nasari_dict.keys() and s2.strip('\n') in nasari_dict.keys():
                test1 = list(map(float, nasari_dict[s1.strip('\n')]))
                test2 = list(map(float, nasari_dict[s2.strip('\n')]))
                sim = cos_sim(test1, test2)
                if sim > max:
                    max = sim
                    max_elem = (s1.strip('\n'), s2.strip('\n'))
    return max, max_elem
                

#### Valutazione accuratezza del sistema

In [ ]:
def get_sense_valutation(user_syn, system_syn):
    i = 0
    unit_score = 0
    couple_score = 0
    for syn in system_syn:
        if syn[0] in user_syn:
            unit_score += 1
        if syn[1] in user_syn:
            unit_score += 1
            
    for syn in system_syn:
        if syn[0] == user_syn[i] and syn[1] == user_syn[i+1]:
            couple_score += 1 
        i += 2
    return f"elementi singoli: {unit_score/100}, coppie: {couple_score/50}"


#### Recupero termini affini da BabelNet

In [ ]:
def get_terms_by_babel_id(babel_id):
    res = []
    x = requests.get(URL.format(babel_id,KEY))
    if x.status_code != 400:
        s = x.json()['senses']
        for sense in s:
            res.append(sense['properties']['fullLemma'])   
    return res

#### Esecuzione

In [ ]:
def mult(res):
    return res*4

result = []
result_elem = []
andrea_syn, fabio_syn = get_synsets_from_txt(PATH_SYN)
andrea_val, fabio_val, mean_val = get_valutation_from_txt(PATH_VAL)

print("INIZIO CONSEGNA 1")
print("---------------------------------------------------------------------------------------")
print(f"\nCorrelazione di Spearman tra annotatori: {stats.spearmanr(andrea_val, fabio_val).correlation}\n")
print(f"\nCorrelazione di Pearson tra annotatori: {stats.pearsonr(andrea_val, fabio_val)}\n")
print("---------------------------------------------------------------------------------------")


for tuple in words:
    syn_1 = get_babel_syns(tuple[0], PATH_SEM)
    syn_2 = get_babel_syns(tuple[1], PATH_SEM)
    
    sim_max, elements = get_nasari_valutation(syn_1, syn_2, nasari_dict)
    
    result.append(sim_max)
    result_elem.append(elements)


print("---------------------------------------------------------------------------------------")
print(f"\nCorrelazione di Spearman tra media annotatori e sistema: {stats.spearmanr(result, mean_val).correlation}\n")
print(f"\nCorrelazione di Pearson tra media annotatori e sistema: {stats.pearsonr(result, mean_val)}\n")
print("---------------------------------------------------------------------------------------")


print("INIZIO CONSEGNA 2")

print(f"\nPunteggio Kappa di Cohen tra annotatori: {cohen_kappa_score(andrea_syn, fabio_syn)}\n")

for elements in result_elem:
    print(f"{tuple}, {elements}, {list(dict.fromkeys(get_terms_by_babel_id(elements[0])))}, {list(dict.fromkeys(get_terms_by_babel_id(elements[1])))}\n")

print("---------------------------------------------------------------------------------------")
print(f"Accuratezza tra sistema e annotazione di Andrea: {get_sense_valutation(andrea_syn, result_elem)}")
print(f"Accuratezza tra sistema e annotazione di Fabio: {get_sense_valutation(fabio_syn, result_elem)}")
print("---------------------------------------------------------------------------------------")
